In [31]:
import sys
sys.path.append('..')

import akshare as ak
import duckdb
import numpy as np
import pandas as pd 

from QHData.query import query
from QHData.data_load import DataLoader,CodesLoader

import warnings

warnings.filterwarnings('ignore')

In [186]:
q = query()

df_stocks = q.fetch_stock_list()    # 获取全市场股票信息
filter_codes = df_stocks[-df_stocks.name.str.startswith(('*',"ST"))]                    # 过滤掉*ST/ST的股票
codes_20cm = filter_codes[filter_codes.code.str.match(r'300|301|688|689')].code.tolist()      # 涨停板为20cm的股票
# --------------------------------------------------------------------------------------

data_loader = DataLoader(start_date='2024-01-01',end_date='2024-12-19',freq='daily')
stk_data = data_loader.load_stock(codes=filter_codes.code.tolist())

Error loading stock data: Executor error during find command :: caused by :: Sort operation used more than the maximum 33554432 bytes of RAM. Add an index, or specify a smaller limit., full error: {'ok': 0.0, 'errmsg': 'Executor error during find command :: caused by :: Sort operation used more than the maximum 33554432 bytes of RAM. Add an index, or specify a smaller limit.', 'code': 96, 'codeName': 'OperationFailed'}


In [193]:
stock_sh_a_spot_em_df = ak.stock_sh_a_spot_em()     # 东方财富网-沪 A 股-实时行情数据
stock_sz_a_spot_em_df = ak.stock_sz_a_spot_em()     # 东方财富网-深 A 股-实时行情数据

# 合并数据
df = pd.concat([stock_sh_a_spot_em_df,stock_sz_a_spot_em_df],axis=0)
df = df[-df['名称'].str.startswith(("*","ST"))]     # 过滤掉*ST/ST的股票
df = df[-df['代码'].str.match(r'300|301|688|689')]  # 过滤掉创业板、科创板

# 过滤掉价格大于10和流通市值大于70的股票
filter_codes = df[(df['最新价'] < 10) & (df['流通市值']/100000000 < 70)]['代码'].tolist()

In [194]:
data_loader = DataLoader(start_date='2023-12-17',end_date='2024-12-23',freq='daily')
stk_data = data_loader.load_stock(codes=filter_codes)
stk_data = stk_data.groupby(level=1).filter(lambda x:len(x) < 252)

In [195]:
# 股价盘整计算
stk_data = stk_data.groupby(level=1).filter(lambda x:x.high.max()/x.low.min() <=2)  
pz = stk_data.groupby(level=1).apply(lambda x:x.close[-1]/(x.high.max() - x.low.min()))
pz_df = pd.DataFrame(data=pz.values,index=pz.index,columns=['盘整'])
pz_df.index.name = '代码'

# 成交量计算
vol_ratio = stk_data.groupby(level=1).apply(lambda x:x.volume[-5:].mean()/x.volume[-10:].mean())
vol_ratio_df = pd.DataFrame(data=vol_ratio.values,index=vol_ratio.index,columns=['均量比'])
vol_ratio_df.index.name = '代码'

In [196]:
# 机构持仓占比

stock_institute_hold_df = ak.stock_institute_hold(symbol="20243")

hold_ratio_dict = {x: stock_institute_hold_df.loc[stock_institute_hold_df['证券代码'] == x, '占流通股比例'].values[0] if x in stock_institute_hold_df['证券代码'].tolist() else 0 for x in stk_data.index.get_level_values(1).unique().to_list()}
hold_ratio_df = pd.DataFrame({'代码':hold_ratio_dict.keys(),'占流通股比例':hold_ratio_dict.values()})

scaled_hold_ratio = 1 / hold_ratio_df[hold_ratio_df['占流通股比例'] != 0]['占流通股比例']
scaled_hold_ratio = 8 * (scaled_hold_ratio - scaled_hold_ratio.min()) / (scaled_hold_ratio.max() - scaled_hold_ratio.min()) + 1
hold_ratio_df.loc[hold_ratio_df['占流通股比例'] != 0, '占流通股比例'] = scaled_hold_ratio
hold_ratio_df[hold_ratio_df['占流通股比例'] != 0]
hold_ratio_df.loc[(hold_ratio_df['占流通股比例']==0),"占流通股比例"] = 10
hold_ratio_df = hold_ratio_df.transform(lambda x:round(x,2))
hold_ratio_df.set_index('代码',inplace=True)


In [197]:

from sklearn.preprocessing import MinMaxScaler

combined_df = pd.concat([pz_df,vol_ratio_df,hold_ratio_df],axis=1)

# 创建 MinMaxScaler 实例
scaler = MinMaxScaler(feature_range=(1, 10))

# 对整个 DataFrame 进行缩放
scaled_data = scaler.fit_transform(combined_df)

# 将缩放后的数据转换回 DataFrame
scaled_df = pd.DataFrame(scaled_data, columns=combined_df.columns, index=combined_df.index)
scaled_df['综合得分'] = scaled_df.sum(axis=1)
scaled_df = scaled_df.sort_values('综合得分',ascending=False).apply(lambda x:round(x,2)).reset_index()


In [198]:
duckdb.sql('SELECT * from scaled_df')

┌─────────┬────────┬────────┬──────────────┬──────────┐
│  代码   │  盘整  │ 均量比 │ 占流通股比例 │ 综合得分 │
│ varchar │ double │ double │    double    │  double  │
├─────────┼────────┼────────┼──────────────┼──────────┤
│ 600681  │   10.0 │   4.11 │         10.0 │    24.11 │
│ 600973  │    3.7 │   10.0 │         10.0 │     23.7 │
│ 000905  │   7.72 │   5.64 │         10.0 │    23.36 │
│ 600354  │   4.32 │   8.23 │         10.0 │    22.55 │
│ 000919  │   3.62 │   8.41 │         10.0 │    22.04 │
│ 000570  │   3.68 │   8.23 │         10.0 │    21.91 │
│ 601008  │   7.11 │   4.68 │         10.0 │    21.79 │
│ 002560  │   4.74 │   6.77 │         10.0 │    21.51 │
│ 002772  │   3.42 │   7.95 │         10.0 │    21.37 │
│ 600359  │   6.08 │   5.17 │         10.0 │    21.24 │
│   ·     │     ·  │     ·  │           ·  │       ·  │
│   ·     │     ·  │     ·  │           ·  │       ·  │
│   ·     │     ·  │     ·  │           ·  │       ·  │
│ 600419  │   3.33 │   2.72 │         1.16 │     7.21 │
│ 603018 

In [185]:
scaled_df.to_csv('20241218.csv')